# Exercice 1:

Pour cet exercice, le problème qui se pose est le suivant :
Trouver une plage horaire de 60 min pendant la semaine à venir pour organiser une réunion entre 8:00 à 17:59.

J'ai commencé la réflexion par une simplification du problème pour mieux visualiser les étapes et complexifier petit à petit jusqu'à ce que le résultat final du code traite le problème dans toute sa complexité.

## Étape 1 : Travail sur une liste d'horaire correspondant au même jour et à la même personne (sans chevauchement)  :

pour cette première étape, j'ai décidé de traiter des listes d'horaire du même jour et sans prendre en compte les chevauchements d'horaire. Comme si mon analyse ne correspond qu'à la recherche du créneau pour une personne unique un jour précis de la semaine.

Pour pouvoir travailler avec la forme imposée par l'énoncé j'ai défini une première fonction __convert__ pour avoir un tuple d'entier correspondant à (heure, minute)

In [1]:
def convert(horaire):
    heure =''
    minute=''
    for i in range(len(horaire)) :
        if horaire[i] ==":":
            break
        heure += horaire[i]
    #on garde que les minutes
    minute= horaire.replace(heure+':','')
    return (int(heure),int(minute))

On fait un test rapide.

In [2]:
horaire="15:20"

convert(horaire)

(15, 20)

Une autre fonction que sera fondamentale pour le trouver une plage de 60 min entre les horaires est la fonction __duree__ qui calcule le nombre de min entre deux horaires.

In [3]:
def duree(horaire1, horaire2):
    
    horaire1=convert(horaire1)
    horaire2=convert(horaire2)
    
    #convertir en minutes
    h1=horaire1[0]*60 +horaire1[1]
    h2=horaire2[0]*60 +horaire2[1]
    duree = h2-h1+1
    return duree


On fait un petit test en faisant bien attention à inclure les minutes de début et de fin dans l'horaire.

In [4]:
horaire1="8:00"
horaire2="8:59"

duree(horaire1, horaire2)

60

Avec une liste contenant un seul horaire on peut faire le choix du créneau de la façon suivante.

In [5]:
L=['1','1 08:30-09:45']
debut="8:00"
fin="17:59"
#tester avant et après le premier créneau
if duree(debut,L[1][2:7])>60 :
    print( "Il y a un horaire entre 8:00 et 8:59 ")
elif duree(L[1][8:],fin)>60 :
    print("Il y a un horaire entre 17:00 et 17:59")


Il y a un horaire entre 17:00 et 17:59


Cette étape nous donne l'idée de tester par la suite entre 8h et le début de la première indisponibilité ainsi que la fin du dernier créneau d'indisponibilité de la même journée et 17:59.

## Étape 2 : Gestion de plusieurs listes d'horaire et des chevauchements

Pour cette deuxième étape nous devons assembler les listes avec les horaires de chaque personne concernées par la réunion et arranger les horaires pour revenir au cas de non-chevauchement des horaires.

Cela nous simplifiera la recherche d'horaire par la suite. Pour l'instant les horaires correspondent toujours à un même jour.

Notre première fonction est une fonction d'ordre qui indique si l'heure 1 et inférieure à l'heure 2 c'est-à-dire plus tôt dans la journée.

In [6]:
def ordre_inferieur(horaire1,horaire2):
    horaire1=convert(horaire1)
    horaire2=convert(horaire2)
    #comparer les heures
    if horaire1[0]< horaire2[0]:
        return True
    #comparer les minutes si les heures sont égales
    elif horaire1[1]<= horaire2[1] and horaire1[0]== horaire2[0]:
        return True 
    else :
        return False

On définit la fonction __ordonner__ qui prend une liste de listes des horaires de chaque personne et la concatène en ordonnant le tout tel que les horaires de début des indisponibilités sont dans l'ordre croissant entre 8: 00 et 17: 59.

Cet ordre nous permettra une implémentation simplifiée de la fonction qui fera la fusion des horaires qui se chevauche. J'ai opté pour le tri à bulles pour sa simplicité.

In [7]:
def ordonner(L_Total):
    #mettre tout dans une même liste
    L=[0]
    for l in L_Total :
        L+=l[1:]
    L[0]=str(len(L[1:])) 
    #trier la liste avec un tri bulle
    for i in range (int(L[0])):
        for j in range (1,int(L[0])-i):
            if ordre_inferieur(L[j+1][2:7],L[j][2:7]):
                L[j+1],L[j]=L[j],L[j+1]
    #retourner la liste triée
    return L

On fait un petit test avec trois listes d'indisponibilité.

In [8]:
L1=['3','1 08:30-09:45','1 10:55-11:30','1 11:10-12:12']
L2=['4','1 08:31-09:50','1 10:45-11:30','1 13:15-14:12','1 17:30-17:40']
L3=['2','1 09:45-10:35','1 14:11-15:35']

L=[L1,L2,L3]

L_ordonnee=ordonner(L)
L_ordonnee

['9',
 '1 08:30-09:45',
 '1 08:31-09:50',
 '1 09:45-10:35',
 '1 10:45-11:30',
 '1 10:55-11:30',
 '1 11:10-12:12',
 '1 13:15-14:12',
 '1 14:11-15:35',
 '1 17:30-17:40']

Maintenant nous allons nous occuper de fusionner les plages d'horaires qui se chevauchent. 

Pour cela j'ai implémenté une première fonction qui teste et fusionne ou pas deux plages horaires.

In [9]:
def fusion(H1,H2):
    L=[]
    #si H2 est inclu dans H1
    if ordre_inferieur(H2[2:7],H1[8:])and ordre_inferieur(H2[8:],H1[8:]):
        L.append(H1)
        return L
    #si H2 debute dans H1 
    elif ordre_inferieur(H1[8:],H2[8:]) and ordre_inferieur(H2[2:7],H1[8:])and ordre_inferieur(H1[2:7],H2[2:7]):
        L.append(H1[0:8]+H2[8:])
        return L
    #si ils sont disjoints
    else:
        L.append(H1)
        L.append(H2)
        return L

In [10]:
h1='1 08:31-09:50'
h2='1 09:45-10:35'
fusion(h1,h2)

['1 08:31-10:35']

Nous allons donc appliquer cette fonction sur notre liste déjà triée.

In [11]:
def fusionner(L):
    #le nombre de creneaux
    n=int(L[0])
    #notre liste où on va retrouver les horaires non superposés
    L_final=[0]
    #On fixe le premier horaire qui nous permet de faire la comparaison au debut puis les changement suivant si il y a eu fusion ou non
    horaireFinRef=L[1]
    for i in range(2,n+1):
        horairefin=L[i]
        
        #si on a pas de fusion 
        if len(fusion(horaireFinRef,horairefin))==2:
            #on rajoute l'horaire s'il n'est pas déja dans la liste
            if horaireFinRef!=L_final[-1]:
                L_final.append(horaireFinRef)
            L_final.append(horairefin)
            #initiation de l'horaire de reference
            horaireFinRef=horairefin
            
       #si on peut fusionner
        else:
            #on insère le nouvel horaire fusionné dans notre liste
            horaireFinRef=fusion(horaireFinRef,horairefin)[0]
            L_final.append(horaireFinRef)
            
            #Si dans notre liste final le nouvel horaire peut se fusionner avec un horaire déjà dans la liste par une fusion anterieure 
            if len(L_final)>2 and len(fusion(L_final[-2],L_final[-1]))==1 :
                #on supprime l'intrevalle le moins englobant.
                del L_final[-2]
            
    L_final[0]=str(len(L_final[1:]))
    return L_final
        

L'implémentation de cette fonction a été un vrai challenge puisque j'ai testé la récursivité en premier temps mais un problème récupération de la liste se posait. C'est pour cette raison que j'ai opté pour une manipulation de liste assez classique avec la récupération des éléments importants dans un élément de référence puis dans une liste.

Nous allons reprendre le test que nous avons effectué précédemment pour voir si notre fonction s'exécute correctement.

In [12]:
L_fusionnee=fusionner(L_ordonnee)
L_fusionnee

['4', '1 08:30-10:35', '1 10:45-12:12', '1 13:15-15:35', '1 17:30-17:40']

On voit bien que le problème est simplifié de cette façon et que maintenant nous pouvons prédire un horaire de disponibilité.


Tout d'abord, j'ai défini une fonction qui donne en sortie l'heure +1 pour une question d'affichage.

In [13]:
def plus_1h(H):
    #convertir en int
    H=convert(H)
    Hbis=[]
    #ajouter une heure en traitant le cas particulier sur les minutes
    if H[1]!=0 :
        Hbis.append(H[0]+1)
        Hbis.append(H[1]-1)
    else:
        Hbis.append(H[0])
        Hbis.append(59)
    #convertir en str
    H=str(Hbis[0])+':'+str(Hbis[1])
    return H

La fonction qui détecte un horaire disponible dans une même journée.

In [14]:
def disponible (L):
    #les horaires limites
    debut="8:00"
    fin="17:59"
    #liste où l'on va mettre l'horaire qui est valable
    horaires=[]
    n=int(L[0])
    i=1
    #on teste le premier horaire
    if duree(debut,L[1][2:7])>60 :
        horaires.append(L[1][0]+' 08:00-08:59')
        
    #on teste entre les horaires de la journée
    while i<n:
        if duree( L[i][8:],L[i+1][2:7])>60 and horaires==[]:
            horaires.append(L[1][0]+' '+L[i][8:]+'-'+plus_1h(L[i][8:]))
        i+=1
    #on teste le dernier 
    if duree(L[-1][8:],fin)>60 and horaires==[]:
            horaires.append(L[1][0]+' 17:00-17:59')
    return horaires

On peux tester sur le même exemple.

Les horaires indisponibles sont :

In [15]:
L_fusionnee

['4', '1 08:30-10:35', '1 10:45-12:12', '1 13:15-15:35', '1 17:30-17:40']

Un horaire de disponibilité :

In [16]:
disponible(L_fusionnee)

['1 12:12-13:11']

__Remarque:__ Pour cette fonction j'ai choisi de rajouter la condition de "horaires==[]" pour n'avoir qu'un seul horaire et ne pas perdre en temps complexité à tester tous les horaires disponibles alors que nous n'avons besoin que d'un seul. J'ai aussi pris le soin de tester l'horaire de fin de journée en dernier parce que je trouve que personne n'apprécie vraiment les réunions à la dernière minute de la journée !

## Etape 3 : Généralisation à tout les jours de la semaine




Pour la généralisation, l'idée est d'utiliser une liste triée d'horaire sans prendre en considération les jours puis la séparer dans une liste de 5 listes correspondant aux 5 jours de la semaine.

Ensuite une liste des disponibilités de chaque jour est calculé à l'aide de la fonction __disponible__ implémentée précédemment.Le premier horaire est donc affiché.

In [17]:
def separer(L):
    L_sep=[[0],[0],[0],[0],[0]]
    #conditions sur les 5 jours de la semaine
    for l in L[1:] :
        if l[0]=='1':
            L_sep[0].append(l)
        elif l[0]=='2':
            L_sep[1].append(l)
        elif l[0]=='3':
            L_sep[2].append(l)
        elif l[0]=='4':
            L_sep[3].append(l)
        else:
            L_sep[4].append(l)
    #initialisation du nombre d'horaires indisponibles dans chaque jour de la semaine     
    for i in range (len(L_sep)):
        L_sep[i][0]=str(len(L_sep[i][1:]))
    return L_sep

In [18]:
def disponible_semaine(L):
    H=[]
    for i in range (len(L)):
        if L[i]!=['0']:
            H.append(disponible(L[i]))
            
        else:
            H.append([str(i)+' 08:00-08:59'])

    i=0
    while i<len(H):
        if H[i]!=[] :
            Horaire=H[i]
            i+=1
            break
    return Horaire

## Test général.

In [20]:
L1=['8','2 08:11-09:23','2 10:45-11:30','1 10:15-12:12','5 14:15-15:18','1 17:30-17:40','4 15:03-16:09','3 14:00-15:17','5 16:19-17:00']
L2=['5','1 10:31-11:50','2 12:35-17:34','1 16:15-17:12','5 11:15-15:18','4 15:30-17:40']
L3=['7','1 08:20-08:50','2 11:45-11:30','1 11:15-12:12','5 14:15-15:18','1 16:30-17:40','3 14:50-17:50','2 16:30-16:55']

#On ordonne par horaire
L_ordre=ordonner([L1,L2,L3])
print("liste ordonée :")
L_ordre

liste ordonée :


['20',
 '2 08:11-09:23',
 '1 08:20-08:50',
 '1 10:15-12:12',
 '1 10:31-11:50',
 '2 10:45-11:30',
 '1 11:15-12:12',
 '5 11:15-15:18',
 '2 11:45-11:30',
 '2 12:35-17:34',
 '3 14:00-15:17',
 '5 14:15-15:18',
 '5 14:15-15:18',
 '3 14:50-17:50',
 '4 15:03-16:09',
 '4 15:30-17:40',
 '1 16:15-17:12',
 '5 16:19-17:00',
 '2 16:30-16:55',
 '1 16:30-17:40',
 '1 17:30-17:40']

In [21]:
#On separe par jour de la semaine
L_sep=separer(L_ordre)
print("liste séparée par jours de la semaine :")
L_sep

liste séparée par jours de la semaine :


[['7',
  '1 08:20-08:50',
  '1 10:15-12:12',
  '1 10:31-11:50',
  '1 11:15-12:12',
  '1 16:15-17:12',
  '1 16:30-17:40',
  '1 17:30-17:40'],
 ['5',
  '2 08:11-09:23',
  '2 10:45-11:30',
  '2 11:45-11:30',
  '2 12:35-17:34',
  '2 16:30-16:55'],
 ['2', '3 14:00-15:17', '3 14:50-17:50'],
 ['2', '4 15:03-16:09', '4 15:30-17:40'],
 ['4', '5 11:15-15:18', '5 14:15-15:18', '5 14:15-15:18', '5 16:19-17:00']]

In [22]:
#on fusionne les horaires dans chaque semaine
L_sep_fusion=[]
for l in L_sep :
    L_sep_fusion.append(fusionner(l))
    
print("la liste des indisponibilités de la semaine séparé par les jours de la semaine :")
L_sep_fusion


la liste des indisponibilités de la semaine séparé par les jours de la semaine :


[['3', '1 08:20-08:50', '1 10:15-12:12', '1 16:15-17:40'],
 ['4', '2 08:11-09:23', '2 10:45-11:30', '2 11:45-11:30', '2 12:35-17:34'],
 ['1', '3 14:00-17:50'],
 ['1', '4 15:03-17:40'],
 ['2', '5 11:15-15:18', '5 16:19-17:00']]

In [23]:
print("on peut faire la réunion à :")
disponible_semaine(L_sep_fusion)

on peut faire la réunion à :


['1 08:50-9:49']